# Lecture 24 – Data 100, Spring 2024

Data 100, Spring 2024

[Acknowledgments Page](https://ds100.org/sp24/acks/)

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.express as px
import seaborn as sns

## Working with High Dimensional Data

In the following cells we will use visualization tools to push as far as we can in visualizing the MPG dataset in high-dimensional space:

In [ ]:
mpg = sns.load_dataset("mpg").dropna()
mpg.head()

In [ ]:
px.histogram(mpg, x="displacement")

In [ ]:
px.scatter(mpg, x="displacement", y="horsepower")

In [ ]:
fig = px.scatter_3d(mpg, x="displacement", y="horsepower", z="weight",
                    width=800, height=800)
fig.update_traces(marker=dict(size=3))

In [ ]:
fig = px.scatter_3d(mpg, x="displacement", 
                    y="horsepower", 
                    z="weight", 
                    color="model_year",
                    width=800, height=800, 
                    opacity=.7)
fig.update_traces(marker=dict(size=5))

In [ ]:
fig = px.scatter_3d(mpg, x="displacement", 
                    y="horsepower", 
                    z="weight", 
                    color="model_year",
                    size="mpg",
                    symbol="origin",
                    width=900, height=800, 
                    opacity=.7)
# hide color scale legend on the plotly fig
fig.update_layout(coloraxis_showscale=False)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2,)

components = pca.fit_transform(mpg[["displacement", "horsepower", "weight", "model_year"]])
mpg[["z1", "z2"]] = components
mpg.head()

In [ ]:
px.scatter(mpg, x="z1", y="z2", color="model_year", hover_data=["displacement", "horsepower", "weight", "name"])

<br><br>

**Return to lecture.**

<br><br>


---

## Singular Value Decomposition 

Singular value decomposition is a numerical technique to automatically decompose matrix into three matrices. Given an input matrix X, SVD will return $U$, $S$ and $V^T$ such that $ X = U S V^T $.

Check the documentation of `np.linalg.svd` [here](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html). There are multiple versions of SVD; to get the version that we will follow, we need to set the `full_matrices` parameter to `False`.

For PCA we will typically work with data that is already centered.

In [ ]:
rectangle = pd.read_csv("data/rectangle_data.csv")
rectangle

In [ ]:
px.scatter_3d(rectangle, x="width", y="height", z="area", 
              width=800, height=800)

In [ ]:
X = rectangle - np.mean(rectangle, axis = 0)
X

In [ ]:
U, S, Vt = np.linalg.svd(X, full_matrices = False)

In [ ]:
print("Shape of U", U.shape)
print("Shape of S", S.shape)
print("Shape of Vt", Vt.shape)

$\Sigma$ is a little different in `NumPy`. Since the only useful values in the diagonal matrix $\Sigma$ are the singular values on the diagonal axis, only those values are returned and they are stored in an array.

Our `rectangle_data` has a rank of $3$, so we should have 3 non-zero singular values, **sorted from largest to smallest**.

In [ ]:
S

Hmm, looks like are four diagonal entries are not zero. What happened?

It turns out there were some numerical rounding errors, but the last value is so small ($10^{-15}$) that it's practically $0$.

In [ ]:
np.isclose(S[3], 0)

In [ ]:
S.round(5)

If we want the diagonal elements:

In [ ]:
Sm = np.diag(S)
Sm

Examining U:

In [ ]:
pd.DataFrame(U).head(5)

Finally, $V^{\top}$:

In [ ]:
pd.DataFrame(Vt)

In [ ]:
Vt.shape

To check that this SVD is a valid decomposition, we can reverse it.

In [ ]:
display(pd.DataFrame(U @ Sm @ Vt).head(5))
display(pd.DataFrame(X).head(5))

## PCA with SVD

### Step 1: Center the Data Matrix $X$

In [ ]:
X = rectangle - np.mean(rectangle, axis = 0)
X.head(10)

In some situations where the units are on different scales it is useful to normalize the data before performing SVD. 
This can be done by dividing each column by its standard deviation.

In [ ]:
Xstd = X / np.std(X, axis = 0)

### Step 2: Get the SVD of centered $X$

In [ ]:
U, S, Vt = np.linalg.svd(X, full_matrices = False)

Examining the singular values:

In [ ]:
pd.DataFrame(np.diag(S))

Computing the contribution to the total variance:

In [ ]:
pd.DataFrame(np.round(S**2 / X.shape[0], 2))

Much of the variance is in the first dimension.  This is likely because the area is much larger than the other dimensions. Let's examine the standardized version.

In [ ]:
U, S, Vt = np.linalg.svd(Xstd, full_matrices = False)

In [ ]:
pd.DataFrame(np.round(S**2 / X.shape[0], 2))

Now we see that most of the variance is in the first two dimensions which makes sense since rectangles are largely described by two numbers.

### Step 3 Computing Approximations to the Data

Let's try to approximate this data in two dimensions

#### Using $Z = U * S$

In [ ]:
Z = U[:, :2] @ np.diag(S[:2])
pd.DataFrame(Z).head()

#### Using $Z = X * V$

In [ ]:
Z = Xstd.to_numpy() @ Vt.T[:,:2]
pd.DataFrame(Z).head()

In [ ]:
px.scatter(x=Z[:, 0], y=Z[:, 1])

Comparing to scikit learn

In [ ]:
pca = PCA(2)
pd.DataFrame(pca.fit_transform(rectangle)).head(5)

In [ ]:
pd.DataFrame(pca.fit_transform(X)).head(5)

In [ ]:
pd.DataFrame(pca.fit_transform(Xstd)).head(5)

Also notice that the covariance of the transformed diagonalized. 

In [ ]:
pd.DataFrame(np.cov(Z.T))

## Lower Rank Approximation of X

Let's now try to recover X from our approximation:

In [ ]:
rectangle.head()

In [ ]:
k = 2
U, S, Vt = np.linalg.svd(Xstd, full_matrices = False)
scaling = np.diag(np.std(X, axis = 0))
# scaling = np.eye(X.shape[1])
Z = U[:,:k] @ np.diag(S[:k])

rectangle_hat = pd.DataFrame(
    (Z @ Vt[:k, :]) @ scaling + np.mean(rectangle, axis = 0).to_numpy(),
    columns = rectangle.columns)

display(rectangle_hat.head(3))

fig = px.scatter_3d(rectangle, x="width", y="height", z="area",
                    width=800, height=800)
fig.add_scatter3d(x=rectangle_hat["width"], y=rectangle_hat["height"], z=rectangle_hat["area"], 
                  mode="markers", name = "approximation")
